In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [4]:
import os
import random
from PIL import Image, ImageEnhance, ImageOps, ImageFilter
import cv2
import numpy as np
import shutil, sys

def copy_txt_file(input_file, output_file):
    """
    Copies the content of the input text file to the output text file.
    """
    shutil.copyfile(input_file, output_file)

def upside_down_augmentation(input_image_jpg, input_text_txt, output_image_path, output_text_path):
    """
    Upside-down augmentation: flips the image and updates the bounding box coordinates accordingly.
    """
    # Load the bounding box coordinates from the text file
    with open(input_text_txt, 'r') as file:
        lines = file.readlines()

    # Load the original image
    img = Image.open(input_image_jpg)

    # Rotate the image and update bounding box coordinates
    img_rotated = img.transpose(method=Image.FLIP_TOP_BOTTOM)
    updated_lines = []

    for line in lines:
        label_data = line.split()
        x_center = float(label_data[1])
        y_center = float(label_data[2])
        box_width = float(label_data[3])
        box_height = float(label_data[4])

        # Calculate the original bounding box coordinates
        x_min = int((x_center - box_width / 2) * img.width)
        y_min = int((y_center - box_height / 2) * img.height)
        x_max = int((x_center + box_width / 2) * img.width)
        y_max = int((y_center + box_height / 2) * img.height)

        # Update the bounding box coordinates by flipping
        updated_y_min = img.height - y_max
        updated_y_max = img.height - y_min

        # Calculate the updated centered coordinates and dimensions
        updated_x_center = (x_min + x_max) / (2 * img.width)
        updated_y_center = (updated_y_min + updated_y_max) / (2 * img.height)
        updated_width = (x_max - x_min) / img.width
        updated_height = (updated_y_max - updated_y_min) / img.height

        # Create the updated line
        updated_line = f"{label_data[0]} {updated_x_center} {updated_y_center} {updated_width} {updated_height}\n"
        updated_lines.append(updated_line)

    # Save the rotated image and updated text file
    img_rotated.save(output_image_path)
    with open(output_text_path, 'w') as file:
        file.writelines(updated_lines)

def left_right_augmentation(input_image_jpg, input_text_txt, output_image_path, output_text_path):
    """
    Left-right augmentation: flips the image horizontally and updates the bounding box coordinates accordingly.
    """
    # Load the bounding box coordinates from the text file
    with open(input_text_txt, 'r') as file:
        lines = file.readlines()

    # Load the original image
    img = Image.open(input_image_jpg)

    # Rotate the image and update bounding box coordinates
    img_rotated = img.transpose(method=Image.FLIP_LEFT_RIGHT)
    updated_lines = []

    for line in lines:
        label_data = line.split()
        x_center = float(label_data[1])
        y_center = float(label_data[2])
        box_width = float(label_data[3])
        box_height = float(label_data[4])

        # Calculate the original bounding box coordinates
        x_min = int((x_center - box_width / 2) * img.width)
        y_min = int((y_center - box_height / 2) * img.height)
        x_max = int((x_center + box_width / 2) * img.width)
        y_max = int((y_center + box_height / 2) * img.height)

        # Update the bounding box coordinates by flipping
        updated_x_min = img.width - x_max
        updated_x_max = img.width - x_min

        # Calculate the updated centroid coordinates and dimensions
        updated_x_center = (updated_x_min + updated_x_max) / (2 * img.width)
        updated_y_center = y_center
        updated_width = (updated_x_max - updated_x_min) / img.width
        updated_height = box_height

        # Create the updated line
        updated_line = f"{label_data[0]} {updated_x_center} {updated_y_center} {updated_width} {updated_height}\n"
        updated_lines.append(updated_line)

    # Save the rotated image and updated text file
    img_rotated.save(output_image_path)
    with open(output_text_path, 'w') as file:
        file.writelines(updated_lines)

def add_noise(img):
    """
    Adds random noise to the image.
    """
    np_img = np.array(img)
    noise = np.random.randint(0, 50, size=np_img.shape, dtype=np.uint8)
    noisy_img = np.clip(np_img + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(noisy_img)

def convert_to_grayscale(img):
    """
    Converts the image to grayscale.
    """
    return img.convert('L')

def adjust_saturation(img, factor):
    """
    Adjusts the saturation of the image.
    """
    enhancer = ImageEnhance.Color(img)
    enhanced_img = enhancer.enhance(factor)
    return enhanced_img

def adjust_brightness(img, factor):
    """
    Adjusts the brightness of the image.
    """
    enhancer = ImageEnhance.Brightness(img)
    enhanced_img = enhancer.enhance(factor)
    return enhanced_img

def adjust_contrast(img, factor):
    """
    Adjusts the contrast of the image.
    """
    enhancer = ImageEnhance.Contrast(img)
    enhanced_img = enhancer.enhance(factor)
    return enhanced_img

def invert_colors(img):
    """
    Inverts the colors of the image.
    """
    return ImageOps.invert(img)

def apply_canny_edge_detection(img):
    """
    Applies Canny edge detection to the image.
    """
    img_gray = img.convert('L')
    img_canny = img_gray.filter(ImageFilter.FIND_EDGES)
    return img_canny

def apply_blur(img):
    """
    Applies blur to the image.
    """
    return img.filter(ImageFilter.BLUR)

def create_augmented_images(image_directory, text_directory, output_image_directory, output_text_directory, num_images_per_input=30):
    """
    Creates augmented images for each input image in the image directory and saves them in the output directories.
    """
    # Get the list of input image files
    image_files = os.listdir(image_directory)
    for image_file in image_files:
        # Construct the input image and text file paths
        input_image_jpg = os.path.join(image_directory, image_file)
        input_text_txt = os.path.join(text_directory, f"{os.path.splitext(image_file)[0]}.txt")

        # Load the original image
        img = Image.open(input_image_jpg)

        # Apply various augmentations to create multiple images
        for i in range(num_images_per_input):
            # Randomly select an augmentation technique
            augmentation_choice = random.choice(['upside_down', 'left_right', 'noise', 'grayscale', 'saturation', 'brightness', 'contrast', 'invert', 'canny_edge', 'blur'])

            # Create a unique filename for the augmented image and text file
            output_image_path = os.path.join(output_image_directory, f"{os.path.splitext(image_file)[0]}_{i}.jpg")
            output_text_path = os.path.join(output_text_directory, f"{os.path.splitext(image_file)[0]}_{i}.txt")

            # Apply the selected augmentation technique
            if augmentation_choice == 'upside_down':
                upside_down_augmentation(input_image_jpg, input_text_txt, output_image_path, output_text_path)
            elif augmentation_choice == 'left_right':
                left_right_augmentation(input_image_jpg, input_text_txt, output_image_path, output_text_path)
            elif augmentation_choice == 'noise':
                augmented_img = add_noise(img)
                augmented_img.save(output_image_path)
                copy_txt_file(input_text_txt, output_text_path)
            elif augmentation_choice == 'grayscale':
                augmented_img = convert_to_grayscale(img)
                augmented_img.save(output_image_path)
                copy_txt_file(input_text_txt, output_text_path)
            elif augmentation_choice == 'saturation':
                saturation_factor = random.uniform(0.5, 1.5)  # Adjust saturation factor as desired
                augmented_img = adjust_saturation(img, saturation_factor)
                augmented_img.save(output_image_path)
                copy_txt_file(input_text_txt, output_text_path)
            elif augmentation_choice == 'brightness':
                brightness_factor = random.uniform(0.5, 1.5)  # Adjust brightness factor as desired
                augmented_img = adjust_brightness(img, brightness_factor)
                augmented_img.save(output_image_path)
                copy_txt_file(input_text_txt, output_text_path)
            elif augmentation_choice == 'contrast':
                contrast_factor = random.uniform(0.5, 1.5)  # Adjust contrast factor as desired
                augmented_img = adjust_contrast(img, contrast_factor)
                augmented_img.save(output_image_path)
                copy_txt_file(input_text_txt, output_text_path)
            elif augmentation_choice == 'invert':
                augmented_img = invert_colors(img)
                augmented_img.save(output_image_path)
                copy_txt_file(input_text_txt, output_text_path)
            elif augmentation_choice == 'canny_edge':
                augmented_img = apply_canny_edge_detection(img)
                augmented_img.save(output_image_path)
                copy_txt_file(input_text_txt, output_text_path)
            elif augmentation_choice == 'blur':
                augmented_img = apply_blur(img)
                augmented_img.save(output_image_path)
                copy_txt_file(input_text_txt, output_text_path)



# Example usage
image_directory = '/content/drive/MyDrive/Manual Augmentation/Annotated_Images'
text_directory = '/content/drive/MyDrive/Manual Augmentation/Annotated_Labels'
output_image_directory = '/content/drive/MyDrive/Manual Augmentation/Augmentated_Images'
output_text_directory = '/content/drive/MyDrive/Manual Augmentation/Augmentated_Text_Files'

# Create the output directories if they don't exist
os.makedirs(output_image_directory, exist_ok=True)
os.makedirs(output_text_directory, exist_ok=True)

# Create augmented images for each input image in the image directory
create_augmented_images(image_directory, text_directory, output_image_directory, output_text_directory)
